In [34]:
from getpass import getpass
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [35]:
# Chrome Setup
driver = webdriver.Chrome()

driver.get("https://www.facebook.com/login")

In [ ]:
# Login with Username and Password
username = driver.find_element('xpath','//*[@id="email"]')
my_username = getpass(prompt="Your Facebook email or phone:")
username.send_keys(my_username)

password = driver.find_element('xpath','//*[@id="pass"]')
my_password = getpass(prompt="Your Password:")
password.send_keys(my_password)

password.send_keys(Keys.RETURN)

## Comments Crawling 

In [12]:
posts = pd.read_csv('posts.csv')
posts

,news,paper,comment_count,id,link
0,0,0,2.6k,0,https://mbasic.facebook.com/DailyProthomAlo/po...
1,0,0,2.1k,1,https://mbasic.facebook.com/story.php?story_fb...
2,0,0,766,2,https://mbasic.facebook.com/DailyProthomAlo/po...
3,0,1,493,3,https://mbasic.facebook.com/BBCBengaliService/...
4,0,1,610,4,https://mbasic.facebook.com/BBCBengaliService/...
...,...,...,...,...,...
126,10,0,202,127,https://mbasic.facebook.com/DailyProthomAlo/po...
127,10,0,104,128,https://mbasic.facebook.com/DailyProthomAlo/po...
128,10,0,33,129,https://mbasic.facebook.com/DailyProthomAlo/po...
129,10,6,1.1k,126,https://mbasic.facebook.com/story.php?story_fb...


# mbasic.facebook.com

In [ ]:
def comment_crawl(i):
	comments = driver.find_elements('xpath','/html/body/div/div/div[2]/div/div[1]/div[2]/div/div/div/div/div[1]')
	# comments = driver.find_elements('xpath','/html/body/div/div/div[2]/div/div[1]/div/div/div[3]/div[2]/div/div/div/div/div/div[1]') # For photo posts
	
	# if len(comments) == 0:
	# 	comments = driver.find_elements_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[4]/div/div/div[1]') [5] for general
	
	for comment in comments:
		cmnt = comment.text.strip()
		if cmnt != '' and cmnt not in comment_texts:
			comment_texts.append(cmnt)
			link_index.append(str(i))

def replies_crawl(i):
	comments = driver.find_elements('xpath','/html/body/div/div/div[2]/div/div[1]/div[3]/div/div/div[1]')
	mentions = driver.find_elements('xpath','/html/body/div/div/div[2]/div/div[1]/div[3]/div/div/div[1]/a')
	
	l = 0
	for comment in comments:
		cmnt = comment.text.strip()
		try:
			mntn = mentions[l].text.strip()
			if mntn in cmnt:
				cmnt = cmnt.replace(mntn, '')
				l+=1
		except:
			pass
		cmnt = cmnt.strip()
		if cmnt != '' and cmnt not in comment_texts:
			comment_texts.append(cmnt)
			link_index.append(str(i))

for i in range(0, 1):
	comment_texts = []
	link_index = []
	link = posts.loc[i]['link']
	driver.get(link)
	
	while(True):
		comment_crawl(i)
		replied_btn = driver.find_elements('xpath','/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div/div/div[4]/div/div/a')
		try:
			for j in range(0, len(replied_btn)):
				replied_btn = driver.find_elements('xpath','/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[5]/div/div/div[4]/div/div/a')
				replied_btn[j].click()
				replies_crawl(i)
				time.sleep(1)
				driver.back()
		except StaleElementReferenceException as Exception:
			pass
		try:
			more_btn = driver.find_element("link text","View more comments…")
			more_btn.click()
			time.sleep(1)
		except NoSuchElementException:
			break
		
	comments_df = pd.DataFrame(zip(link_index,comment_texts))
	comments_df.to_csv('comments.csv', mode='a', index=False, header=False)

In [12]:
data = pd.read_csv('comments.csv', error_bad_lines=False)
print(data.count())

C:\Users\sajal\AppData\Local\Temp/ipykernel_24112/3413172473.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('comments.csv', error_bad_lines=False)
b'Skipping line 134205: expected 2 fields, saw 3\n'


LinkIndex    136584
Comment      136583
dtype: int64


In [4]:
data_csv = data.sort_values(by=['LinkIndex'])
data_csv.to_csv('comments_final.csv', index=False)